# **Check results**

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import json

In [ ]:
SEEDS = ['42', '1234', '1867', '613', '1001']
BETAS = ["10", "90", "50", "MIX"]
RESULTS_ROOT = "../03_results"

In [ ]:
def get_outcomes(meas_name):
    meas_path = f"{RESULTS_ROOT}/{meas_name}"
    parking_events = pd.read_xml(f"{meas_path}/stop_data.out.xml", xpath="stopinfo")
    parking_events = parking_events.sort_values(by="started", ignore_index=True)
    
    with open(f"{meas_path}/auction_results.json") as f:
        auction_results = json.load(f)
    
    prices = []
    distances = []
    auction_prices = []
    garages = 0

    for ar in auction_results["outcomes"]:
        parking_filt = parking_events[parking_events["id"] == ar["vehicle"]]
        parking_filt = parking_filt[parking_filt["started"] >= ar["time"]]
        if len(parking_filt) > 0:
            if "garage" in ar["auction_id"]:
                garages += 1
            else:
                pe = parking_filt.iloc[0]
                prices.append(np.ceil((pe.ended - pe.started)/3600)*ar["price"])
                auction_prices.append(ar["price"])
                distances.append(ar["distance"])
                
    answer = {
        "prices": prices,
        "auction_prices": auction_prices,
        "distances": distances,
        "garages": garages,
        "n_auctions": len(auction_results["outcomes"])
    }
    return answer

In [ ]:
BETA = 10

In [ ]:
prices, distances, auction_prices = [], [], []
garages = []

for seed in SEEDS:
    outcomes = get_outcomes(f"beta{BETA}_{seed}")
    prices = prices + outcomes["prices"]
    distances = distances + outcomes["distances"]
    auction_prices = auction_prices + outcomes["auction_prices"]
    garages = garages + [outcomes["garages"]/outcomes["n_auctions"]]

In [ ]:
plt.scatter(distances, prices, alpha=0.25, marker="o", label="curbside")
plt.legend()

In [ ]:
np.mean(distances), np.mean(prices), np.mean(auction_prices), np.mean(garages)